In [1]:
import pymongo
from pprint import pprint

mongo_client = pymongo.MongoClient("mongodb://localhost:27018/")
mongo_db = mongo_client["depictioDB"]

workflow_name = "ashleys-qc-pipeline"
collection = mongo_db[workflow_name]
data_sources = collection.distinct("metadata.report_general_stats_headers", {"wf_name": workflow_name})
pprint(data_sources)


[{'error_rate': {'ceiling': None,
                 'colour': '55,126,184',
                 'description': 'Error rate: mismatches (NM) / bases mapped '
                                '(CIGAR)',
                 'dmax': 100.0,
                 'dmin': 0.0,
                 'floor': None,
                 'format': '{:,.2f}',
                 'hidden': None,
                 'max': 100,
                 'min': 0,
                 'minRange': None,
                 'modify': 100.0,
                 'namespace': 'Samtools',
                 'placement': 1000.0,
                 'rid': 'mqc-generalstats-samtools-error_rate',
                 'scale': 'OrRd',
                 'shared_key': None,
                 'suffix': '%',
                 'title': 'Error rate'},
  'non-primary_alignments': {'ceiling': None,
                             'colour': '55,126,184',
                             'description': 'Non-primary alignments (millions)',
                             'dmax': 2.277565,

In [14]:
import pandas as pd
df = pd.DataFrame.from_records([d[key] | {"key": key} for d in data_sources for key in d])
df.groupby("namespace")["description"].unique().to_dict()

,title,description,min,max,suffix,scale,format,modify,namespace,rid,colour,hidden,ceiling,floor,minRange,shared_key,placement,dmax,dmin,key
0,Error rate,Error rate: mismatches (NM) / bases mapped (CI...,0,100.0,%,OrRd,"{:,.2f}",100.000000,Samtools,mqc-generalstats-samtools-error_rate,"55,126,184",None,None,None,None,None,1000.0,100.000000,0.0,error_rate
1,M Non-Primary,Non-primary alignments (millions),0,NaN,NaN,PuBu,"{:,.1f}",0.000001,Samtools,mqc-generalstats-samtools-non_primary_alignments,"55,126,184",None,None,None,None,read_count,1000.0,2.277565,0.0,non-primary_alignments
2,M Reads Mapped,Reads Mapped in the bam file (millions),0,NaN,NaN,GnBu,"{:,.1f}",0.000001,Samtools,mqc-generalstats-samtools-reads_mapped,"55,126,184",None,None,None,None,read_count,1000.0,2.277565,0.0,reads_mapped
3,% Mapped,% Mapped Reads,0,100.0,%,RdYlGn,"{:,.1f}",NaN,Samtools,mqc-generalstats-samtools-reads_mapped_percent,"55,126,184",None,None,None,None,None,1000.0,100.000000,0.0,reads_mapped_percent
4,% Proper Pairs,% Properly Paired Reads,0,100.0,%,RdYlGn,"{:,.1f}",NaN,Samtools,mqc-generalstats-samtools-reads_properly_paire...,"55,126,184",False,None,None,None,None,1000.0,100.000000,0.0,reads_properly_paired_percent
5,% MapQ 0 Reads,% of Reads that are Ambiguously Placed (MapQ=0),0,100.0,%,OrRd,"{:,.1f}",NaN,Samtools,mqc-generalstats-samtools-reads_MQ0_percent,"55,126,184",True,None,None,None,None,1000.0,100.000000,0.0,reads_MQ0_percent
6,M Total seqs,Total sequences in the bam file (millions),0,NaN,NaN,GnBu,"{:,.1f}",0.000001,Samtools,mqc-generalstats-samtools-raw_total_sequences,"55,126,184",None,None,None,None,read_count,1000.0,2.277565,0.0,raw_total_sequences
7,Error rate,Error rate: mismatches (NM) / bases mapped (CI...,0,100.0,%,OrRd,"{:,.2f}",100.000000,Samtools,mqc-generalstats-samtools-error_rate,"55,126,184",None,None,None,None,None,1000.0,100.000000,0.0,error_rate
8,M Non-Primary,Non-primary alignments (millions),0,NaN,NaN,PuBu,"{:,.1f}",0.000001,Samtools,mqc-generalstats-samtools-non_primary_alignments,"55,126,184",None,None,None,None,read_count,1000.0,2.558834,0.0,non-primary_alignments
9,M Reads Mapped,Reads Mapped in the bam file (millions),0,NaN,NaN,GnBu,"{:,.1f}",0.000001,Samtools,mqc-generalstats-samtools-reads_mapped,"55,126,184",None,None,None,None,read_count,1000.0,2.558834,0.0,reads_mapped


In [18]:
df.groupby("namespace")["description"].unique().to_dict()

{'FastQC': array(['% Duplicate Reads', 'Average % GC Content',
        'Average Read Length (bp)', 'Median Read Length (bp)',
        'Percentage of modules failed in FastQC report (includes those not plotted here)',
        'Total Sequences (millions)'], dtype=object),
 'Samtools': array(['Error rate: mismatches (NM) / bases mapped (CIGAR)',
        'Non-primary alignments (millions)',
        'Reads Mapped in the bam file (millions)', '% Mapped Reads',
        '% Properly Paired Reads',
        '% of Reads that are Ambiguously Placed (MapQ=0)',
        'Total sequences in the bam file (millions)',
        'Total reads in the bam file (millions)',
        '% Reads Mapped in the bam file'], dtype=object)}

In [9]:
dev_col = mongo_db["dev"]
dev_col.insert_many(df.to_dict("records"))

In [10]:
# Query data from MongoDB and convert it to Arrow table
cursor = dev_col.find({})
df = pd.DataFrame(list(cursor))

# Convert Arrow table back to pandas DataFrame
df

,_id,title,description,min,max,suffix,scale,format,modify,namespace,...,colour,hidden,ceiling,floor,minRange,shared_key,placement,dmax,dmin,key
0,6438737175e1b1fe946f8a75,Error rate,Error rate: mismatches (NM) / bases mapped (CI...,0,100.0,%,OrRd,"{:,.2f}",100.000000,Samtools,...,"55,126,184",None,None,None,None,None,1000.0,100.000000,0.0,error_rate
1,6438737175e1b1fe946f8a76,M Non-Primary,Non-primary alignments (millions),0,NaN,NaN,PuBu,"{:,.1f}",0.000001,Samtools,...,"55,126,184",None,None,None,None,read_count,1000.0,2.277565,0.0,non-primary_alignments
2,6438737175e1b1fe946f8a77,M Reads Mapped,Reads Mapped in the bam file (millions),0,NaN,NaN,GnBu,"{:,.1f}",0.000001,Samtools,...,"55,126,184",None,None,None,None,read_count,1000.0,2.277565,0.0,reads_mapped
3,6438737175e1b1fe946f8a78,% Mapped,% Mapped Reads,0,100.0,%,RdYlGn,"{:,.1f}",NaN,Samtools,...,"55,126,184",None,None,None,None,None,1000.0,100.000000,0.0,reads_mapped_percent
4,6438737175e1b1fe946f8a79,% Proper Pairs,% Properly Paired Reads,0,100.0,%,RdYlGn,"{:,.1f}",NaN,Samtools,...,"55,126,184",False,None,None,None,None,1000.0,100.000000,0.0,reads_properly_paired_percent
5,6438737175e1b1fe946f8a7a,% MapQ 0 Reads,% of Reads that are Ambiguously Placed (MapQ=0),0,100.0,%,OrRd,"{:,.1f}",NaN,Samtools,...,"55,126,184",True,None,None,None,None,1000.0,100.000000,0.0,reads_MQ0_percent
6,6438737175e1b1fe946f8a7b,M Total seqs,Total sequences in the bam file (millions),0,NaN,NaN,GnBu,"{:,.1f}",0.000001,Samtools,...,"55,126,184",None,None,None,None,read_count,1000.0,2.277565,0.0,raw_total_sequences
7,6438737175e1b1fe946f8a7c,Error rate,Error rate: mismatches (NM) / bases mapped (CI...,0,100.0,%,OrRd,"{:,.2f}",100.000000,Samtools,...,"55,126,184",None,None,None,None,None,1000.0,100.000000,0.0,error_rate
8,6438737175e1b1fe946f8a7d,M Non-Primary,Non-primary alignments (millions),0,NaN,NaN,PuBu,"{:,.1f}",0.000001,Samtools,...,"55,126,184",None,None,None,None,read_count,1000.0,2.558834,0.0,non-primary_alignments
9,6438737175e1b1fe946f8a7e,M Reads Mapped,Reads Mapped in the bam file (millions),0,NaN,NaN,GnBu,"{:,.1f}",0.000001,Samtools,...,"55,126,184",None,None,None,None,read_count,1000.0,2.558834,0.0,reads_mapped
